In [2]:
import os
import random
from tqdm.notebook import tqdm
from config import BaseConfig
from models.utils import load_pkl, load_json, save_pkl, save_json
import warnings
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report

warnings.filterwarnings('ignore')
CONFIG = BaseConfig().get_args()
label_encoder = load_pkl(CONFIG.path_saved_le)

In [3]:
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

print(f"device is: {device}")

device is: cuda


In [5]:
train_data = load_json(CONFIG.train_path)
test_data = load_json(CONFIG.test_path)
dev_data = load_json(CONFIG.dev_path)

train_rel = [[example['tail']['word'], example['head']['word'], example['relation']]
             for example in tqdm(train_data)]

dev_rel = [[example['tail']['word'], example['head']['word'], example['relation']]
           for example in tqdm(dev_data)]

test_rel = [[example['tail']['word'], example['head']['word'], example['relation']]
            for example in tqdm(test_data)]

  0%|          | 0/534277 [00:00<?, ?it/s]

  0%|          | 0/114506 [00:00<?, ?it/s]

  0%|          | 0/114565 [00:00<?, ?it/s]

In [6]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 50
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 2
LEARNING_RATE = 1e-05
distilbert_path = 'assets/transformers/distilbert-base-uncased'

tokenizer = DistilBertTokenizer.from_pretrained(distilbert_path)

In [7]:
class BioRel(Dataset):
    def __init__(self, data, tokenizer, max_len, label_encoder):
        self.len = len(data)
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_encoder = label_encoder
        
    def __getitem__(self, index):
        tail = " ".join(self.data[index][0].split())
        head = " ".join(self.data[index][1].split())
        target = self.label_encoder.transform([self.data[index][2]])[0]
        
        inputs_tail = self.tokenizer.encode_plus(
            tail,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        inputs_head = self.tokenizer.encode_plus(
            head,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs_tail['input_ids'] + inputs_head['input_ids'] 
        mask = inputs_tail['attention_mask'] + inputs_head['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(target, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [8]:
train_dataset = BioRel(train_rel, tokenizer, MAX_LEN, label_encoder)
dev_dataset = BioRel(dev_rel, tokenizer, MAX_LEN, label_encoder)
test_dataset = BioRel(test_rel, tokenizer, MAX_LEN, label_encoder)

In [9]:
train_params = {
    'batch_size': TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
}

val_params = {
    'batch_size': VALID_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
}

test_params = { 
    'batch_size': VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
}


train_loader = DataLoader(train_dataset, **train_params)
dev_loader = DataLoader(dev_dataset, **val_params)
test_loader = DataLoader(test_dataset, **test_params)

In [10]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self, C, path):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained(path)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, C)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [11]:
model = DistillBERTClass(C=len(label_encoder.classes_), path=distilbert_path)
model.to(device)

Some weights of the model checkpoint at assets/transformers/distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat

In [12]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [13]:
# Function to calcuate the accuracy of the model
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [15]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    y_predict = []
    y_true = []
    for _,data in tqdm(enumerate(train_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)
        for y_pred in big_idx.cpu().numpy():
            y_predict.append(y_pred)
        for y_target in targets.cpu().numpy():
            y_true.append(y_target)
        
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%10000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 10000 steps: {loss_step}")
            print(f"Training Accuracy per 10000 steps: {accu_step}")
            print("-------------------------------------------------------------------")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")
    print("========================================================")
    print(f"Epoch Evaluations {epoch}: \n")
    print(evaluation_method(y_true, y_predict))
    print("========================================================")

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

Training Loss per 10000 steps: 4.853753089904785
Training Accuracy per 10000 steps: 0.0
-------------------------------------------------------------------


# Save Model

In [18]:
path_to_save_model = "assets/model/distilbert-finetuned"

output_model_file = os.path.join(path_to_save_model, "pytorch_distilbert_biorel.pth")
torch.save(model.state_dict(), output_model_file)


# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.load_state_dict(copy.deepcopy(torch.load("model_state.pth",device)))                                  

print('All files saved')

All files saved
This tutorial is completed


# Inference

In [21]:
path_to_save_model = "assets/model/distilbert-finetuned"
model_file = os.path.join(path_to_save_model, "pytorch_distilbert_biorel.pth")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model2 = DistillBERTClass(C=len(label_encoder.classes_), path=distilbert_path)

model2.load_state_dict(torch.load(model_file,device))


model2.to(device)

Some weights of the model checkpoint at assets/transformers/distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat